In [ ]:
!pip install -q crewai langchain langchain_core langchain_community langchain-openai

In [14]:
!pip install -q langchain-experimental wikipedia "wikibase-rest-api-client<0.2" mediawikiapi arxiv

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.6/103.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 6.0 MB/s eta 0:00:00


In [9]:
from pydantic import BaseModel, Field
from typing import List, Optional, Dict

class CodeInput(BaseModel):
    code_snippet: str
    language: str = Field(default="python", description="Programming language of the code snippet")
    context: Optional[str] = Field(default=None, description="Additional context or comments about the code")

class FunctionElement(BaseModel):
    name: str
    signature: str
    docstring: Optional[str]

class ClassElement(BaseModel):
    name: str
    signature: str
    docstring: Optional[str]
    methods: List[FunctionElement]

class ParsingOutput(BaseModel):
    functions: List[FunctionElement]
    classes: List[ClassElement]
    modules: Optional[List[str]]

class FunctionDocumentation(BaseModel):
    function_name: str
    description: str
    parameters: List[Dict[str, str]]
    return_type: Optional[str]
    examples: Optional[List[str]]

class ClassDocumentation(BaseModel):
    class_name: str
    description: str
    methods: List[FunctionDocumentation]
    attributes: Optional[List[Dict[str, str]]]
    examples: Optional[List[str]]

class ModuleDocumentation(BaseModel):
    module_name: str
    description: str
    functions: List[FunctionDocumentation]
    classes: List[ClassDocumentation]
    examples: Optional[List[str]]

class DocumentationOutput(BaseModel):
    documentation: str
    module_documentation: ModuleDocumentation

In [10]:
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [16]:
from crewai import Agent
from textwrap import dedent
from langchain_openai import ChatOpenAI
from langchain_experimental.tools import PythonREPLTool
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools.wikidata.tool import WikidataAPIWrapper, WikidataQueryRun
from langchain_community.utilities import ArxivAPIWrapper
from langchain.tools import Tool

class CustomAgents:
    def __init__(self):
        self.OpenAIGPT4Mini = ChatOpenAI(model="gpt-4o-mini", temperature=0)
        self.OpenAIGPT4 = ChatOpenAI(model="gpt-4o", temperature=0)
        self.tools = [WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper()),
                 WikidataQueryRun(api_wrapper=WikidataAPIWrapper()),
                 Tool(
                     name="Arxiv",
                     func=ArxivAPIWrapper().run,
                     description="A wrapper around Arxiv. Useful for when you need to access academic papers."
                 )]

    def code_parser_agent(self):
        # Agent 1: Code Parser
        return Agent(
            role="Code Parser",
            backstory=dedent("""You are an expert in parsing code to extract functions, classes, and modules."""),
            goal=dedent("""Parse the provided code and extract all functions, classes, and modules along with their signatures."""),
            tools=self.tools,
            allow_delegation=False,
            verbose=True,
            llm=self.OpenAIGPT4Mini,
        )

    def documentation_writer_agent(self):
        # Agent 2: Documentation Writer
        return Agent(
            role="Documentation Writer",
            backstory=dedent("""You specialize in writing detailed documentation for code elements such as functions, classes, and modules."""),
            goal=dedent("""Write comprehensive documentation for each extracted code element, including descriptions, parameters, return types, and usage examples."""),
            tools=self.tools,
            allow_delegation=False,
            verbose=True,
            llm=self.OpenAIGPT4,
        )

    def examples_generator_agent(self):
        # Agent 3: Examples Generator
        return Agent(
            role="Examples Generator",
            backstory=dedent("""You provide practical usage examples for code elements to demonstrate how they can be used."""),
            goal=dedent("""Generate usage examples for each code element to help users understand how to use them in practice."""),
            tools=self.tools,
            allow_delegation=False,
            verbose=True,
            llm=self.OpenAIGPT4Mini,
        )

    def final_assembler_agent(self):
        # Agent 4: Final Assembler
        tools = []
        return Agent(
            role="Final Assembler",
            backstory=dedent("""You assemble all the documentation pieces into a final, cohesive documentation output."""),
            goal=dedent("""Compile all the documentation and examples into a well-structured documentation file in the desired format."""),
            tools=tools,
            allow_delegation=False,
            verbose=True,
            llm=self.OpenAIGPT4,
        )


In [17]:
from crewai import Task
from textwrap import dedent
import json

class CustomTasks:
    def __init__(self):
        pass

    def code_parsing_task(self, agent, code_input: CodeInput):
        parsing_output_schema = ParsingOutput.schema_json(indent=2)
        return Task(
            description=dedent(f"""
                Parse the following {code_input.language} code snippet and extract all functions, classes, and modules along with their signatures and docstrings.
                Provide your output in **JSON format** matching the **ParsingOutput** schema.

                **Format**:
                ```json
                {parsing_output_schema}
                ```

                **Code**:
                ```{code_input.language}
                {code_input.code_snippet}
                ```

                **Additional Context**:
                {code_input.context if code_input.context else 'N/A'}
            """),
            agent=agent,
            expected_output=f"The parsing output in JSON format matching the schema: {parsing_output_schema}",
        )

    def documentation_writing_task(self, agent, code_input: CodeInput):
        documentation_output_schema = DocumentationOutput.schema_json(indent=2)
        return Task(
            description=dedent(f"""
                Based on the parsed code elements provided, write comprehensive documentation for each function, class, and module.
                Include descriptions, parameters, return types, and any other relevant details.
                Provide your output in **JSON format** matching the **DocumentationOutput** schema.

                **Format**:
                ```json
                {documentation_output_schema}
                ```

                **Parsed Elements**:
                (Please use the parsing output from the previous task.)

                **Additional Context**:
                {code_input.context if code_input.context else 'N/A'}
            """),
            agent=agent,
            expected_output=f"The documentation output in JSON format matching the schema: {documentation_output_schema}",
        )

    def examples_generation_task(self, agent, code_input: CodeInput):
        examples_output_schema = {
            "examples": [{"element_name": "str", "example_code": "str"}]
        }
        examples_output_schema_json = json.dumps(examples_output_schema, indent=2)

        return Task(
            description=dedent(f"""
                Generate usage examples for each code element extracted.
                Provide your output in **JSON format** matching the **ExamplesOutput** schema.

                **Format**:
                ```json
                {examples_output_schema_json}
                ```

                **Code Elements**:
                (Please use the parsing output from the previous task.)

                **Additional Context**:
                {code_input.context if code_input.context else 'N/A'}
            """),
            agent=agent,
            expected_output=f"The examples output in JSON format matching the schema: {examples_output_schema_json}",
        )

    def documentation_assembly_task(self, agent, code_input: CodeInput):
        final_documentation_schema = {
            "documentation": "str"
        }
        final_documentation_schema_json = json.dumps(final_documentation_schema, indent=2)

        return Task(
            description=dedent(f"""
                Assemble all the documentation and examples into a well-structured documentation file.
                Ensure the documentation is clear, comprehensive, and follows best practices.
                Provide your output in **JSON format** matching the **FinalDocumentation** schema.

                **Format**:
                ```json
                {final_documentation_schema_json}
                ```

                **Documentation Components**:
                (Include the documentation and examples from the previous tasks.)

                **Additional Context**:
                {code_input.context if code_input.context else 'N/A'}
            """),
            agent=agent,
            expected_output=f"The final documentation in JSON format matching the schema: {final_documentation_schema_json}",
        )


In [18]:
import os
from crewai import Crew
from textwrap import dedent
import json

class DocumentationGeneratorCrew:
    def __init__(self, code_snippet, language="python", context=None):
        self.code_input = CodeInput(code_snippet=code_snippet, language=language, context=context)
        self.agents = CustomAgents()
        self.tasks = CustomTasks()

    def run(self):
        # Define agents
        code_parser_agent = self.agents.code_parser_agent()
        documentation_writer_agent = self.agents.documentation_writer_agent()
        examples_generator_agent = self.agents.examples_generator_agent()
        final_assembler_agent = self.agents.final_assembler_agent()

        # Define tasks
        code_parsing_task = self.tasks.code_parsing_task(code_parser_agent, self.code_input)
        documentation_writing_task = self.tasks.documentation_writing_task(documentation_writer_agent, self.code_input)
        examples_generation_task = self.tasks.examples_generation_task(examples_generator_agent, self.code_input)
        documentation_assembly_task = self.tasks.documentation_assembly_task(final_assembler_agent, self.code_input)

        # Create the crew
        crew = Crew(
            agents=[
                code_parser_agent,
                documentation_writer_agent,
                examples_generator_agent,
                final_assembler_agent,
            ],
            tasks=[
                code_parsing_task,
                documentation_writing_task,
                examples_generation_task,
                documentation_assembly_task,
            ],
            verbose=True,
        )

        result = crew.kickoff()
        return result

if __name__ == "__main__":
    print("## Welcome to the Documentation Generator Assistant")
    print("-------------------------------------------------------")
    code_snippet = input(dedent("""Please enter the code you wish to generate documentation for:\n"""))
    language = input("Enter the programming language (default is Python): ") or "python"
    context = input("Any additional context or comments about the code? ") or None

    crew = DocumentationGeneratorCrew(code_snippet, language, context)
    results = crew.run()
    print("\n\n########################")
    print("## Here is your documentation generation result:")
    print("########################\n")
    print(results)


## Welcome to the Documentation Generator Assistant
-------------------------------------------------------
Please enter the code you wish to generate documentation for:
import torch import torch.nn as nn import torch.optim as optim import torchvision.transforms as transforms import torchvision.datasets as datasets from torch.utils.data import DataLoader import numpy as np import matplotlib.pyplot as plt  # Configuración de dispositivo (GPU si está disponible) device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 1. Definimos la arquitectura de la CNN en un módulo separado class CNN(nn.Module):     def __init__(self):         super(CNN, self).__init__()         # Primera capa convolucional + max pooling         self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)         self.pool = nn.MaxPool2d(kernel_size=2, stride=2)                  # Segunda capa convolucional + max pooling         self.conv2 = nn.Conv2d(in_channels=32, out_channels=6

# Agent: Code Parser
## Task: 
                Parse the following python code snippet and extract all functions, classes, and modules along with their signatures and docstrings.
                Provide your output in **JSON format** matching the **ParsingOutput** schema.

                **Format**:
                ```json
                {
  "$defs": {
    "ClassElement": {
      "properties": {
        "name": {
          "title": "Name",
          "type": "string"
        },
        "signature": {
          "title": "Signature",
          "type": "string"
        },
        "docstring": {
          "anyOf": [
            {
              "type": "string"
            },
            {
              "type": "null"
            }
          ],
          "title": "Docstring"
        },
        "methods": {
          "items": {
            "$ref": "#/$defs/FunctionElement"
          },
          "title": "Methods",
          "type": "array"
        }
      },
      "required": [
        "na

In [19]:
results

CrewOutput(raw='```json\n{\n  "documentation": "This documentation provides a comprehensive guide to implementing a Convolutional Neural Network (CNN) architecture using the MNIST dataset. The following sections detail the various components and functions used in the implementation, along with example code snippets to illustrate their usage.\\n\\n1. **get_data_loaders**\\n   - **Description**: This function is responsible for loading the MNIST dataset and preparing the data loaders for training and validation.\\n   - **Example**:\\n     ```python\\n     train_loader, val_loader = get_data_loaders(batch_size=32)\\n     ```\\n\\n2. **CNN**\\n   - **Description**: This class defines the CNN architecture used for classifying the MNIST dataset.\\n   - **Example**:\\n     ```python\\n     model = CNN()\\n     ```\\n\\n3. **forward**\\n   - **Description**: This method performs the forward pass of the CNN model.\\n   - **Example**:\\n     ```python\\n     output = model.forward(input_tensor)\